In [1]:
import os
import time
import torch
import pandas as pd
import numpy as np
from torch.optim import Adam
from accelerate import Accelerator
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, Sampler
from transformers import T5EncoderModel, T5Tokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix
from ProtTransModel2 import ClassConfig, T5EncoderCLSModel, Contrastive_loss, LayerLoss, NodeLoss
from accelerate.logging import get_logger
from sklearn.metrics import roc_curve, auc, matthews_corrcoef
import json
import math
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from MySampler2 import MyDistributedSampler

/home/csy/miniconda3/envs/deepspeed/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/csy/miniconda3/envs/deepspeed/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/csy/miniconda3/envs/deepspeed/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit 

In [4]:
model_dirname = '/step_Focalloss6_'
model_path = '/data/csyData/prot_t5_xl_half_uniref50-enc'

with open('/data/csyData/uniprot_test/data/XML_testdata/go_temp/GO_data/cco/label_freq_list.json') as f:
    label_freq_list = json.load(f)

path = '/data/csyData/uniprot_test/data/XML_testdata/go_temp/GO_data/cco/withlayer2/'
datatype = 'stride/'
train_data_dir1 = path + datatype + 'traindata.csv'
train_data_dir2 = path + datatype + 'trainNolabel.csv'
# train_data_dir3 = path + 'traindata2.csv'
train_filelist = [train_data_dir1, train_data_dir2]


test_data_dir1 = path + datatype + 'testdata.csv'
test_data_dir2 = path + datatype + 'testNolabel.csv'
# test_data_dir3 = path + 'testdata2.csv'
test_filelist = [test_data_dir1, test_data_dir2]

# 得到 根 -> 叶子 字典
with open('/data/csyData/uniprot_test/data/XML_testdata/go_temp/GO_data/cco/GO_cco_withlayerDepthDict3.json') as f:
    temp_hierar_relations = json.load(f)
    for go in temp_hierar_relations:
        temp_hierar_relations[go] = temp_hierar_relations[go]['leaf']

# 得到 分层聚类字典
with open('/data/csyData/pygosemsim-master/pygosemsim-master/cco_Kmeans_cluster2.json') as f:
    temp_cluster_nodes = json.load(f)

# 得到 聚类的父子关系字典
# with open('/data/csyData/uniprot_test/code/GOcode/cco_version2/cco_cluster_relations.json') as f:
with open('/data/csyData/pygosemsim-master/pygosemsim-master/cco_KMeanscluster_relations.json') as f:
    cluster_relations = json.load(f)

# 得到标签频率按照阈值分成四份的字典
# with open('/data/csyData/uniprot_test/data/XML_testdata/go_temp/GO_data/cco/label_freq_list.json') as f:
#     label_freq_list = json.load(f)

# 得到标签频率按照字典
# with open('/data/csyData/uniprot_test/data/XML_testdata/go_temp/GO_data/cco/sta_count3.json') as f:
#     sta_count3 = json.load(f)

# 得到alpha
with open('/data/csyData/uniprot_test/code/GOcode/cco_version2/freq.json') as f:
    freq = json.load(f)

node_nums = len(temp_hierar_relations.keys()) - 1
layer_nums = len(temp_cluster_nodes.keys())
labels_num = node_nums + layer_nums

epoch = 200    # 可能修改
max_length = 512
train_batch_size = 64
valid_batch_size = 64
# lr = 3e-4
# lr = 1e-3
only_layer_metrics = False
# load_model = True
# saved_ep = 50   # 重新训练时，记得改成最新的模型参数
saved_ep = 0

temp_go = list(pd.read_csv(test_data_dir2, nrows=0).columns)[2:]

high_freq = {}
low_freq = {}
hierar_relations = {}

def go2index(target_go):
    go = temp_go
    if isinstance(target_go, list):
        return [go.index(i) for i in target_go]
    elif isinstance(target_go, str):
        if target_go in go:
            return go.index(target_go)
        else:
            return None
        
for node,value in temp_hierar_relations.items():
    if go2index(node):
        hierar_relations[str(go2index(node))] = go2index(value)

main_numbers, sub_numbers = [], []
for label in list(temp_cluster_nodes.keys()):
    if '_' not in label:
        main_numbers.append(label)
    else:
        sub_numbers.append(label)

main_numbers.sort(key=lambda x: int(x))
sub_numbers.sort(key=lambda x: (int(x.split('_')[0]), int(x.split('_')[1])) if '_' in x else (int(x), -1))
cluster_nodes = main_numbers + sub_numbers


cluster_nodes_relations = {}
for index,value in enumerate(cluster_nodes):
    cluster_nodes_relations[str(index)] = go2index(temp_cluster_nodes[value])

alpha = [0] * node_nums

In [5]:
model = T5EncoderModel.from_pretrained(model_path)
model.config.d_model = 1024
tokenizer = T5Tokenizer.from_pretrained(model_path)

lora_target_modules = []
for name, _ in model.named_modules():
    for layer in [23]:
        if (str(layer) in name) & (('.wi' in name) or ('.wo' in name) or ('.q' in name) or ('.k' in name) or ('.v' in name) or ('.o' in name)):
            lora_target_modules.append(name)

peft_config = LoraConfig(
                lora_alpha=16,
                lora_dropout=0.1,
                r=8,
                bias="none",
                task_type="SEQ_2_SEQ_LM",
                target_modules=lora_target_modules
                )
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

class_config = ClassConfig(node_nums=node_nums, layer_nums=layer_nums, batch_size=train_batch_size)

class_model = T5EncoderCLSModel(model.config,
                                class_config,
                                cluster_relations,
                                hierar_relations,
                                cluster_nodes_relations,
                                alpha)
class_model.encoder = model.encoder

model = class_model
del class_model
model = model.cuda()

/home/csy/miniconda3/envs/deepspeed/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
from safetensors.torch import load_file

saved_path = './ckpts6/step_1600/model/model.safetensors'


weights = load_file(saved_path)

# 现在 weights 是一个字典，其中包含了模型的所有权重
# 你可以将这些权重加载到你的模型中
model.load_state_dict(weights)

<All keys matched successfully>

In [9]:
model

T5EncoderCLSModel(
  (shared): Embedding(128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
              (dropout): Dr

In [ ]:
def collate_func(batch):
    sequence, layers, nodes = [], [], []
    for item in batch:
        sequence.append(item[0])
        layers.append(item[1])
        nodes.append(item[2])
    inputs = tokenizer(sequence, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    # del inputs["token_type_ids"]
    if not only_layer_metrics:
        inputs["nodes"] = torch.FloatTensor(np.stack(nodes)).to('cuda:1')
    inputs["layers"] = torch.FloatTensor(np.stack(layers)).to('cuda:1')
    return inputs

class MyDataLoader(DataLoader):
    def __init__(self, *args, **kwargs):
        super(MyDataLoader, self).__init__(*args, **kwargs)
        self.go = self.dataset.go

    def go2index(self,target_go):
        if isinstance(target_go, list):
            return [self.go.index(i) for i in target_go]
        elif isinstance(target_go, str):
            if target_go in self.go:
                return self.go.index(target_go)
            else:
                return None

class MyDataset(Dataset):

    def __init__(self,data_dirlist) -> None:
        super().__init__()
        self.data = pd.concat(data_dirlist,axis=0).reset_index(drop=True)
        self.go = list(self.data.columns)[2:]
        
        # low_freq_label_indices = self.data[self.data[list(low_freq.keys())].eq(1).any(axis=1)].index.tolist()
        # temp = self.data.loc[low_freq_label_indices]
        # temp['sequence'] = temp['sequence'].apply(lambda x: x[::-1])
        # self.data = pd.concat([self.data] + [temp]).reset_index(drop=True)

        # self.high_freq_label_indices = self.data[self.data[list(high_freq.keys())].eq(1).any(axis=1) & self.data[list(low_freq.keys())].eq(0).all(axis=1)].index.tolist()
        # self.low_freq_label_indices = self.data[self.data[list(low_freq.keys())].eq(1).any(axis=1)].index.tolist()

        # self.other_indices = self.data.index.difference(self.low_freq_label_indices).difference(self.high_freq_label_indices)

    def __getitem__(self, index):
        temp_nodelabel = np.array(self.data.iloc[index,2:].tolist())
        temp_layerlabel = np.array([])
        
        for cluster_name in cluster_nodes:
            golist = temp_cluster_nodes[cluster_name]
            temp_layerlabel = np.append(temp_layerlabel, 
                                        np.sum(temp_nodelabel[np.array(self.go2index(golist))]) > 0)
        return self.data["sequence"].iloc[index], temp_layerlabel, temp_nodelabel
    
    def __len__(self):
        return len(self.data)


validdata_list = []
for i in test_filelist:
    validdata_list.append(pd.read_csv(i))
validset = MyDataset(validdata_list)

valid_sampler = MyDistributedSampler(validset, \
                                    num_replicas=1, \
                                    rank=0, \
                                    shuffle=True, \
                                    initial_ratio=0.5, \
                                    drop_last=False)


validloader = MyDataLoader(validset, batch_size=valid_batch_size, collate_fn=collate_func, sampler=valid_sampler)